In [214]:
from bs4 import BeautifulSoup

from getpass import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
import time
import scrapy
import datetime
import unidecode
import re
import numpy as np
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://nomadlist.com/")


In [152]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [14]:
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

In [160]:
### City Name
city = list(map(lambda name: name.text, driver.find_elements_by_class_name('itemName')))

In [159]:
### Country
country = list(map(lambda cont: cont.text, driver.find_elements_by_class_name('itemSub')))

In [158]:
### Ranking
rank = list(map(lambda rk: rk.text, driver.find_elements_by_class_name('rank')))

In [157]:
### Cost of living
cost = list(map(lambda ct: ct.text, driver.find_elements_by_class_name('price')))

In [19]:
### Description
desc = list(map(lambda de: de.text, driver.find_elements_by_class_name('description')))

In [161]:
### update lists 
### delete 2 position from city
city.remove('') 
city.remove('Visit Asia')

### delete from country
country.remove('')

### delete from cost
cost.remove('')
del cost[1348:]

ValueError: list.remove(x): x not in list

In [162]:
### Check Lengths
print('cost ', len(cost))
print('rank ', len(rank))
print('country ', len(country))
print('city ', len(city))

cost  1348
rank  1348
country  1348
city  1348


In [163]:
### Create DF
nomadlist = pd.DataFrame({
        'Rank': rank,
        'City': city, 
        'Country': country, 
        'Cost of Living': cost, 
    })

In [172]:
### Save to pickle
#nomadlist.to_pickle('NomadList.pickle')

In [216]:
#Read pickle
nomadlist = pd.read_pickle(r'C:\Users\frans\Documents\GitHub\Project-4_WebScraping\NomadList.pickle')
nomadlist

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\frans\\Documents\\GitHub\\Project-4_WebScraping\\NomadList.pickle'

## 2 -Edit Nomadlist dtypes

In [73]:
nomadlist

,Rank,City,Country,Cost of Living
0,1,Lisbon,Portugal,"$3,464"
1,2,Chiang Mai,Thailand,"$1,315"
2,3,"Canggu, Bali",Indonesia,"$2,013"
3,4,Bangkok,Thailand,"$2,090"
4,5,Madeira,Portugal,"$2,085"
...,...,...,...,...
1343,1344,Dar es Salaam,Tanzania,"$1,746"
1344,1345,Saint Petersburg,Russia,"$2,384"
1345,1346,Yangon,Myanmar,"$1,330"
1346,1347,Baghdad,Iraq,"$1,904"


In [3]:
#Read pickle
nomadlist = pd.read_pickle(r'C:\Users\frans\Documents\GitHub\Project-4_WebScraping\NomadList.pickle')
nomadlist
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.strip('$')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.replace(',','')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].astype(int)

In [85]:
#Save to pickle
#nomadlist.to_pickle('nomad.pickle')

## 3 - Scrap ranking from secondary page & pass the column 'city' as url 

In [4]:
### Create df to pass city names as page url
nameurl = nomadlist[['City','Rank']]
citypage = nameurl['City'].str.split(',', expand = True)
citypage[0] = citypage[0].str.lower()
citypage[0] = citypage[0].str.replace(' ','-')
citypage.rename(columns = {0:'city'}, inplace=True)

In [8]:
citypage

,city,1
0,lisbon,None
1,chiang-mai,None
2,canggu,Bali
3,bangkok,None
4,madeira,None
...,...,...
1343,dar-es-salaam,None
1344,saint-petersburg,None
1345,yangon,None
1346,baghdad,None


In [7]:
#save to csv
citypage.to_csv('citypage.csv')

In [93]:
citypage = pd.read_csv('/Users/anacarolinaquintino/Documents/GitHub/Project-4_WebScraping/EDA/citypage.csv')

In [96]:
### Scrap multiple pages 
citylist = citypage.city.tolist()

def get_url(page):
    return 'https://nomadlist.com/{}'.format(page)

url_list = list(map(get_url,citylist))
url_list


['https://nomadlist.com/lisbon',
 'https://nomadlist.com/chiang-mai',
 'https://nomadlist.com/canggu',
 'https://nomadlist.com/bangkok',
 'https://nomadlist.com/madeira',
 'https://nomadlist.com/buenos-aires',
 'https://nomadlist.com/mexico-city',
 'https://nomadlist.com/timisoara',
 'https://nomadlist.com/berlin',
 'https://nomadlist.com/bengaluru',
 'https://nomadlist.com/gran-canaria',
 'https://nomadlist.com/cape-town',
 'https://nomadlist.com/medellín',
 'https://nomadlist.com/penang',
 'https://nomadlist.com/portimão',
 'https://nomadlist.com/lagos',
 'https://nomadlist.com/fuerteventura',
 'https://nomadlist.com/ho-chi-minh-city',
 'https://nomadlist.com/porto',
 'https://nomadlist.com/kuala-lumpur',
 'https://nomadlist.com/tenerife',
 'https://nomadlist.com/ko-pha-ngan',
 'https://nomadlist.com/brasov',
 'https://nomadlist.com/nairobi',
 'https://nomadlist.com/kathmandu',
 'https://nomadlist.com/istanbul',
 'https://nomadlist.com/ubud',
 'https://nomadlist.com/varna',
 'https:/

In [128]:
##DF with all urls & empty column for results
url_df = pd.DataFrame(url_list).reset_index().rename(columns = {0 : 'url'})
url_df['results'] = url_df.apply(lambda city_url : '', axis=1)
url_df
#url_df.head(50)

,index,url,results
0,0,https://nomadlist.com/lisbon,
1,1,https://nomadlist.com/chiang-mai,
2,2,https://nomadlist.com/canggu,
3,3,https://nomadlist.com/bangkok,
4,4,https://nomadlist.com/madeira,
...,...,...,...
1343,1343,https://nomadlist.com/dar-es-salaam,
1344,1344,https://nomadlist.com/saint-petersburg,
1345,1345,https://nomadlist.com/yangon,
1346,1346,https://nomadlist.com/baghdad,


/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_10583/3118036702.py:24: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


6  seconds of time used for getting  ['1']
5  seconds of time used for getting  ['3']
6  seconds of time used for getting  ['4']
7  seconds of time used for getting  ['2']
6  seconds of time used for getting  ['5']
6  seconds of time used for getting  ['6']
4  seconds of time used for getting  ['7']
4  seconds of time used for getting  ['8']
5  seconds of time used for getting  ['11']
4  seconds of time used for getting  ['13']
4  seconds of time used for getting  ['9']
5  seconds of time used for getting  ['10']
7  seconds of time used for getting  empty
4  seconds of time used for getting  ['14']
5  seconds of time used for getting  empty
6  seconds of time used for getting  ['217']
7  seconds of time used for getting  ['17']
4  seconds of time used for getting  ['24']
5  seconds of time used for getting  ['12']
5  seconds of time used for getting  ['20']
5  seconds of time used for getting  ['19']
7  seconds of time used for getting  ['22']
6  seconds of time used for getting  ['23'

NameError: name 'driver' is not defined

In [139]:
### Define Web scrapping (WS) function
options = Options()
options.add_argument("window-size=1400,600")
#ua = UserAgent()
#user_agent = ua.random
#print(user_agent)
#options.add_argument(f'user-agent={user_agent}')
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
    ua = UserAgent()
    user_agent = ua.random
    #print(user_agent)
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get(url)
    #x = random.randint(4,7)
    time.sleep(1)
    rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(result)
    driver.close()
    return rating01

#final = list(map(all_process,url_list))
#driver.close()
#url_df['results'] = url_df.apply(lambda x_url: all_process(x_url['url']) if x_url['results'] == '' else x_url, axis=1)
#url_df

In [ ]:
## 1st run of the WS function
url_df['results'] = url_df.apply(lambda x_url: all_process(x_url['url']) if x_url['results'] == '' else x_url, axis=1)
url_df
#url_df.to_csv('url_results.csv')

In [ ]:
## Do not consider
### Web scrapping rank
#options = Options()
#options.add_argument("window-size=1400,600")
#ua = UserAgent()
#user_agent = ua.random
#print(user_agent)
#options.add_argument(f'user-agent={user_agent}')
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
#def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
 #   ua = UserAgent()
  #  user_agent = ua.random
    #print(user_agent)
   # options.add_argument(f'user-agent={user_agent}')
    #driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    #driver.get(url)
    #x = random.randint(4,7)
    #time.sleep(x)
    #rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    #result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    #print(x,' seconds of time used for getting ', result)
    #return rating01

#final = list(map(all_process,url_list))
#url_df['results'] = url_df.apply(lambda x_url: all_process(x_url['url']) if x_url['results'] == '' else x_url, axis=1)
#driver.close()
#url_df

In [5]:
## Do not consider
### Web scrapping rank
#useragent = UserAgent()
#profile = webdriver.FirefoxProfile()
#profile.set_preference("general.useragent.override", useragent.random)
#driver = webdriver.Firefox(FirefoxDriverManager().install())
#driver = webdriver.Firefox(firefox_profile=profile, executable_path="C:\\Utility\\BrowserDrivers\\geckodriver.exe")

#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
#def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
 #   driver.get(url)
 #   x = random.randint(4,7)
  #  time.sleep(x)
   # rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    #result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    #print(x,' seconds of time used for getting ', result)
    #return result

#final = list(map(all_process,url_list[:20]))
#driver.close()
#final

In [180]:
url_results = pd.read_csv('/Users/anacarolinaquintino/Documents/GitHub/Project-4_WebScraping/EDA/url_results.csv')

In [181]:
## Fix Urls with the correct city names
url_results['url'] = url_results['url'].apply(unidecode.unidecode) ##remove all special characters of the letters
url_results['url'] = url_results['url'].str.replace("'",'')
url_results['url'] = url_results['url'].str.replace('st.-','st-')
url_results.head(20)

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_1917/3066732940.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  url_results['url'] = url_results['url'].str.replace('st.-','st-')


,Unnamed: 0,index,url,results
0,0,0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...
1,1,1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...
2,2,2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...
3,3,3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...
4,4,4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...
5,5,5,https://nomadlist.com/buenos-aires,['⭐️ Total score\n3.85/5 (Rank #7)\n921 review...
6,6,6,https://nomadlist.com/mexico-city,[]
7,7,7,https://nomadlist.com/timisoara,['⭐️ Total score\n3.63/5 (Rank #10)\n70 review...
8,8,8,https://nomadlist.com/berlin,['⭐️ Total score\n3.6/5 (Rank #12)\n1952 revie...
9,9,9,https://nomadlist.com/bengaluru,['⭐️ Total score\n3.62/5 (Rank #11)\n147 revie...


In [183]:
## 2nd run of the WS function with the correct city names
url_results['results'] = url_results.apply(lambda x_url: all_process(x_url['url']) if len(x_url['results']) == 2 else x_url['results'], axis=1)
url_results.to_csv('url_results_'+str(datetime.datetime.now())+'.csv')
url_results

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_1917/2091384232.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


['7']
['14']
['13']
['16']
['30']
empty
['45']
['58']
['97']
['102']
['117']
['1138']
['221']
['206']
['189']
['1006']
['246']
['287']
['340']
['225']
['314']
['342']
['346']
['385']
['315']
['347']
['381']
['360']
empty
['442']
['390']
['399']
['394']
['457']
['520']
['461']
['1120']
['454']
['485']
['513']
['500']
['548']
['561']
['539']
['597']
['570']
['599']
['662']
['750']
['604']
['563']
['659']
['605']
['648']
['785']
['793']
['720']
['859']
['836']
['727']
['765']
['789']
['876']
['823']
['762']
['851']
['754']
['701']
['788']
['818']
['736']
['931']
['683']
['848']
['910']
['877']
['165']
['808']
['982']
['900']
['948']
['973']
['968']
['991']
['1014']
['1031']
['1005']
['1109']
['1142']
['1079']
['994']
['1103']
['1137']
['1097']
['1181']
['367']
empty
empty
['1214']
['1216']
['1219']
['1245']
['1265']
['1272']
['1280']
['1276']
['1283']
['1290']
['1300']
['1302']
['1303']
['1342']
['1353']
['1309']
['1330']


,Unnamed: 0,index,url,results
0,0,0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...
1,1,1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...
2,2,2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...
3,3,3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...
4,4,4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...
...,...,...,...,...
1343,1343,1343,https://nomadlist.com/dar-es-salaam,['⭐️ Total score\n0.5/5 (Rank #1356)\n48 revie...
1344,1344,1344,https://nomadlist.com/saint-petersburg,['⭐️ Total score\n0.5/5 (Rank #1323)\n314 revi...
1345,1345,1345,https://nomadlist.com/yangon,['⭐️ Total score\n0.5/5 (Rank #1348)\n109 revi...
1346,1346,1346,https://nomadlist.com/baghdad,['⭐️ Total score\n0.5/5 (Rank #1328)\n6 review...


In [184]:
url_results_1 = pd.read_csv('/Users/anacarolinaquintino/Documents/GitHub/Project-4_WebScraping/EDA/url_results_2022-12-11 14:45:31.416823.csv')

In [185]:
## 3rd run of the WS function to get the outstanding url info 
url_results_1['results'] = url_results_1.apply(lambda x_url: all_process(x_url['url']) if len(x_url['results']) == 2 else x_url['results'], axis=1)
url_results_1.to_csv('url_results_'+str(datetime.datetime.now())+'.csv')
url_results_1

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_1917/2091384232.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


['46']
['364']
['1193']
['1168']


,Unnamed: 0.1,Unnamed: 0,index,url,results
0,0,0,0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...
1,1,1,1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...
2,2,2,2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...
3,3,3,3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...
4,4,4,4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...
...,...,...,...,...,...
1343,1343,1343,1343,https://nomadlist.com/dar-es-salaam,['⭐️ Total score\n0.5/5 (Rank #1356)\n48 revie...
1344,1344,1344,1344,https://nomadlist.com/saint-petersburg,['⭐️ Total score\n0.5/5 (Rank #1323)\n314 revi...
1345,1345,1345,1345,https://nomadlist.com/yangon,['⭐️ Total score\n0.5/5 (Rank #1348)\n109 revi...
1346,1346,1346,1346,https://nomadlist.com/baghdad,['⭐️ Total score\n0.5/5 (Rank #1328)\n6 review...


## Final df (includes all the info for all urls) 

In [198]:
url_results_1 = url_results_1[['url', 'results']] 
url_results_1

,url,results
0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...
1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...
2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...
3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...
4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...
...,...,...
1343,https://nomadlist.com/dar-es-salaam,['⭐️ Total score\n0.5/5 (Rank #1356)\n48 revie...
1344,https://nomadlist.com/saint-petersburg,['⭐️ Total score\n0.5/5 (Rank #1323)\n314 revi...
1345,https://nomadlist.com/yangon,['⭐️ Total score\n0.5/5 (Rank #1348)\n109 revi...
1346,https://nomadlist.com/baghdad,['⭐️ Total score\n0.5/5 (Rank #1328)\n6 review...


## 5. Get Score and Rank

In [201]:
#type(url_results['results'][0])
url_results_1[['result1','result2']] = url_results_1['results'].str.split('Rank', expand=True)
url_results_1

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_1917/2184077496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_results_1[['result1','result2']] = url_results_1['results'].str.split('Rank', expand=True)


,url,results,result1,result2
0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...,['⭐️ Total score\n4.34/5 (,#1)\n2384 reviews\n❤️ Liked by members\n👍86% ...
1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...,['⭐️ Total score\n4.06/5 (,#3)\n1583 reviews\n❤️ Liked by members\n👍91% ...
2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...,['⭐️ Total score\n4.04/5 (,#4)\n1287 reviews\n❤️ Liked by members\n👍81% ...
3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...,['⭐️ Total score\n4.08/5 (,#2)\n2250 reviews\n❤️ Liked by members\n👍89% ...
4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...,['⭐️ Total score\n3.88/5 (,#5)\n275 reviews\n❤️ Liked by members\n👍100% ...
...,...,...,...,...
1343,https://nomadlist.com/dar-es-salaam,['⭐️ Total score\n0.5/5 (Rank #1356)\n48 revie...,['⭐️ Total score\n0.5/5 (,#1356)\n48 reviews\n👍 Quality of life score\n...
1344,https://nomadlist.com/saint-petersburg,['⭐️ Total score\n0.5/5 (Rank #1323)\n314 revi...,['⭐️ Total score\n0.5/5 (,#1323)\n314 reviews\n❤️ Liked by members\n👍10...
1345,https://nomadlist.com/yangon,['⭐️ Total score\n0.5/5 (Rank #1348)\n109 revi...,['⭐️ Total score\n0.5/5 (,#1348)\n109 reviews\n❤️ Liked by members\n👍10...
1346,https://nomadlist.com/baghdad,['⭐️ Total score\n0.5/5 (Rank #1328)\n6 review...,['⭐️ Total score\n0.5/5 (,#1328)\n6 reviews\n👍 Quality of life score\nB...


In [212]:
url_results_1['score'] = url_results_1['result1'].apply(lambda x : re.findall('([0-9].[0-9]+)/5', str(x)))
url_results_1['score'] = url_results_1['score'].apply(lambda x : ''.join(x))
url_results_1['rank'] = url_results_1['result2'].apply(lambda x : re.findall('#([0-9]+)', str(x)))
url_results_1['rank'] = url_results_1['rank'].apply(lambda x : ''.join(x))
url_results_1

,url,results,result1,result2,score,rank
0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...,['⭐️ Total score\n4.34/5 (,#1)\n2384 reviews\n❤️ Liked by members\n👍86% ...,4.34,1
1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...,['⭐️ Total score\n4.06/5 (,#3)\n1583 reviews\n❤️ Liked by members\n👍91% ...,4.06,3
2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...,['⭐️ Total score\n4.04/5 (,#4)\n1287 reviews\n❤️ Liked by members\n👍81% ...,4.04,4
3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...,['⭐️ Total score\n4.08/5 (,#2)\n2250 reviews\n❤️ Liked by members\n👍89% ...,4.08,2
4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...,['⭐️ Total score\n3.88/5 (,#5)\n275 reviews\n❤️ Liked by members\n👍100% ...,3.88,5
...,...,...,...,...,...,...
1343,https://nomadlist.com/dar-es-salaam,['⭐️ Total score\n0.5/5 (Rank #1356)\n48 revie...,['⭐️ Total score\n0.5/5 (,#1356)\n48 reviews\n👍 Quality of life score\n...,0.5,1356
1344,https://nomadlist.com/saint-petersburg,['⭐️ Total score\n0.5/5 (Rank #1323)\n314 revi...,['⭐️ Total score\n0.5/5 (,#1323)\n314 reviews\n❤️ Liked by members\n👍10...,0.5,1323
1345,https://nomadlist.com/yangon,['⭐️ Total score\n0.5/5 (Rank #1348)\n109 revi...,['⭐️ Total score\n0.5/5 (,#1348)\n109 reviews\n❤️ Liked by members\n👍10...,0.5,1348
1346,https://nomadlist.com/baghdad,['⭐️ Total score\n0.5/5 (Rank #1328)\n6 review...,['⭐️ Total score\n0.5/5 (,#1328)\n6 reviews\n👍 Quality of life score\nB...,0.5,1328


In [215]:
nomadlist

NameError: name 'nomadlist' is not defined

In [13]:
import re
import numpy as np
import random
### into df

def all_process(url):
    html = requests.get(url)
    print(html)
    x = random.randint(10,20)
    time.sleep(x)
    soup = BeautifulSoup(html.text, 'html.parser')
    rating01 = list(map(lambda joke: joke.get_text(), soup.find_all(class_ = 'details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return result

final = list(map(all_process,url_list[:2]))
final

<Response [200]>
19  seconds of time used for getting  ['1']
<Response [503]>
11  seconds of time used for getting  empty
<Response [200]>
14  seconds of time used for getting  ['4']
<Response [200]>
19  seconds of time used for getting  ['2']
<Response [200]>
10  seconds of time used for getting  ['5']
<Response [503]>
12  seconds of time used for getting  empty
<Response [200]>
16  seconds of time used for getting  ['7']
<Response [200]>
20  seconds of time used for getting  ['8']
<Response [200]>
11  seconds of time used for getting  ['9']
<Response [200]>
15  seconds of time used for getting  ['12']


[['1'], 'empty', ['4'], ['2'], ['5'], 'empty', ['7'], ['8'], ['9'], ['12']]